In [1]:
!pip install python-crfsuite


     |████████████████████████████████| 749 kB 2.6 MB/s eta 0:00:01


In [2]:
from itertools import chain
import pycrfsuite
import sklearn
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

In [3]:
import codecs

class CorpusReader(object):

    def __init__(self, path):
        with codecs.open(path, encoding='utf-8') as f:
            sent = []
            sents = []
            for line in f:
                if line == '\n':
                    sents.append(sent)
                    sent = []
                    continue
                morph_info = line.strip().split('\t')
                sent.append(morph_info)
        train_num = int(len(sents) * 0.9)
        self.__train_sents = sents[:train_num]
        self.__test_sents = sents[train_num:]

    def iob_sents(self, name):
        if name == 'train':
            return self.__train_sents
        elif name == 'test':
            return self.__test_sents
        else:
            return None

In [4]:
c = CorpusReader('hironsan.txt')
train_sents = c.iob_sents('train')
test_sents = c.iob_sents('test')

In [5]:
train_sents[0]

[['2005', '名詞', '数', '*', '*', '*', '*', '*', 'B-DAT'],
 ['年', '名詞', '接尾', '助数詞', '*', '*', '*', '年', 'ネン', 'ネン', 'I-DAT'],
 ['7', '名詞', '数', '*', '*', '*', '*', '*', 'I-DAT'],
 ['月', '名詞', '一般', '*', '*', '*', '*', '月', 'ツキ', 'ツキ', 'I-DAT'],
 ['14', '名詞', '数', '*', '*', '*', '*', '*', 'I-DAT'],
 ['日', '名詞', '接尾', '助数詞', '*', '*', '*', '日', 'ニチ', 'ニチ', 'I-DAT'],
 ['、', '記号', '読点', '*', '*', '*', '*', '、', '、', '、', 'O'],
 ['南アフリカ',
  '名詞',
  '固有名詞',
  '地域',
  '国',
  '*',
  '*',
  '南アフリカ',
  'ミナミアフリカ',
  'ミナミアフリカ',
  'B-LOC'],
 ['共和', '名詞', '一般', '*', '*', '*', '*', '共和', 'キョウワ', 'キョーワ', 'I-LOC'],
 ['国', '名詞', '接尾', '一般', '*', '*', '*', '国', 'コク', 'コク', 'I-LOC'],
 ['ダーバン', '名詞', '固有名詞', '地域', '一般', '*', '*', 'ダーバン', 'ダーバン', 'ダーバン', 'I-LOC'],
 ['で', '助詞', '格助詞', '一般', '*', '*', '*', 'で', 'デ', 'デ', 'O'],
 ['開催', '名詞', 'サ変接続', '*', '*', '*', '*', '開催', 'カイサイ', 'カイサイ', 'O'],
 ['中', '名詞', '接尾', '副詞可能', '*', '*', '*', '中', 'チュウ', 'チュー', 'O'],
 ['の', '助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 

In [6]:
def is_hiragana(ch):
    return 0x3040 <= ord(ch) <= 0x309F

def is_katakana(ch):
    return 0x30A0 <= ord(ch) <= 0x30FF

def get_character_type(ch):
    if ch.isspace():
        return 'ZSPACE'
    elif ch.isdigit():
        return 'ZDIGIT'
    elif ch.islower():
        return 'ZLLET'
    elif ch.isupper():
        return 'ZULET'
    elif is_hiragana(ch):
        return 'HIRAG'
    elif is_katakana(ch):
        return 'KATAK'
    else:
        return 'OTHER'

def get_character_types(string):
    character_types = map(get_character_type, string)
    character_types_str = '-'.join(sorted(set(character_types)))

    return character_types_str

In [7]:
def extract_pos_with_subtype(morph):
    idx = morph.index('*')

    return '-'.join(morph[1:idx])

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    chtype = get_character_types(sent[i][0])
    postag = extract_pos_with_subtype(sent[i])
    features = [
        'bias',
        'word=' + word,
        'type=' + chtype,
        'postag=' + postag,
    ]
    if i >= 2:
        word2 = sent[i-2][0]
        chtype2 = get_character_types(sent[i-2][0])
        postag2 = extract_pos_with_subtype(sent[i-2])
        iobtag2 = sent[i-2][-1]
        features.extend([
            '-2:word=' + word2,
            '-2:type=' + chtype2,
            '-2:postag=' + postag2,
            '-2:iobtag=' + iobtag2,
        ])
    else:
        features.append('BOS')

    if i >= 1:
        word1 = sent[i-1][0]
        chtype1 = get_character_types(sent[i-1][0])
        postag1 = extract_pos_with_subtype(sent[i-1])
        iobtag1 = sent[i-1][-1]
        features.extend([
            '-1:word=' + word1,
            '-1:type=' + chtype1,
            '-1:postag=' + postag1,
            '-1:iobtag=' + iobtag1,
        ])
    else:
        features.append('BOS')

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        chtype1 = get_character_types(sent[i+1][0])
        postag1 = extract_pos_with_subtype(sent[i+1])
        features.extend([
            '+1:word=' + word1,
            '+1:type=' + chtype1,
            '+1:postag=' + postag1,
        ])
    else:
        features.append('EOS')

    if i < len(sent)-2:
        word2 = sent[i+2][0]
        chtype2 = get_character_types(sent[i+2][0])
        postag2 = extract_pos_with_subtype(sent[i+2])
        features.extend([
            '+2:word=' + word2,
            '+2:type=' + chtype2,
            '+2:postag=' + postag2,
        ])
    else:
        features.append('EOS')

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [morph[-1] for morph in sent]


def sent2tokens(sent):
    return [morph[0] for morph in sent]

In [9]:
sent2features(train_sents[0])[0]

['bias',
 'word=2005',
 'type=ZDIGIT',
 'postag=名詞-数',
 'BOS',
 'BOS',
 '+1:word=年',
 '+1:type=OTHER',
 '+1:postag=名詞-接尾-助数詞',
 '+2:word=7',
 '+2:type=ZDIGIT',
 '+2:postag=名詞-数']

In [10]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [11]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [12]:
trainer.train('model.crfsuite')


In [22]:
tagger = pycrfsuite.Tagger()
tagger.open('model.crfsuite')

In [45]:
example_sent = test_sents[48]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))


x = sent2labels(example_sent)

for i in range(len(x)):
    if x[i]== 'B-PSN':
        s = sent2tokens(example_sent)
        print('名前：'+s[i])

地元 の LBC テレビ は 当初 、 主 に 飛散 し た ガラス の 破片 により 6 名 が 負傷 し 、 乗用車 の 所有 者 は 地元 の 住民 ヨセフ・ナディム た と 報じ た 。
Predicted: O O B-ORG I-ORG O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
Correct:   O O B-ORG I-ORG O O O O O O O O O O O O O O O O O O O O O O O O O O B-PSN O O O O O
名前：ヨセフ・ナディム


In [53]:
example_sent = test_sents[48]
print('入力データ： ' + ' '.join(sent2tokens(example_sent)))

x = sent2labels(example_sent)

for i in range(len(x)):
    if x[i]== 'B-PSN':
        s = sent2tokens(example_sent)
        print('名前：'+s[i]+ '(正解データ)')                

for i in range(len(x)):
    if x[i]== 'B-PSN':
        s = tagger.tag(sent2features(example_sent))
        print('名前：'+s[i]+ '(予測値)')        

入力データ： 地元 の LBC テレビ は 当初 、 主 に 飛散 し た ガラス の 破片 により 6 名 が 負傷 し 、 乗用車 の 所有 者 は 地元 の 住民 ヨセフ・ナディム た と 報じ た 。
名前：ヨセフ・ナディム(正解データ)
名前：O(予測値)


In [39]:
for i in range(100):
    example_sent = test_sents[i]
    print(' '.join(sent2tokens(example_sent)))

    print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))

昨年 10 月 に は 、 34 人 が 、 今回 の 現場 に 近い エジプト の タバ で 爆発 事件 の ため 死亡 し て いる 。
Predicted: B-DAT I-DAT I-DAT O O O O O O O O O O O O B-LOC O B-LOC O O O O O O O O O O
Correct:   B-DAT I-DAT I-DAT O O O O O O O O O O O O B-LOC O B-LOC O O O O O O O O O O
タバ は イスラエル から の 観光 客 に 人気 の ある リゾート 地 で ある 。
Predicted: B-LOC O B-LOC O O O O O O O O O O O O O
Correct:   B-LOC O B-LOC O O O O O O O O O O O O O
この ため 、 当局 は 今回 の 爆発 は イスラエル ・ パレスチナ 間 の 緊張 に かかわる もの として 非難 し て いる 。
Predicted: O O O O O O O O O B-LOC O B-LOC O O O O O O O O O O O O
Correct:   O O O O O O O O O B-LOC O B-LOC O O O O O O O O O O O O
地元 の 警察 に よれ ば 少なくとも 30 人 が 死亡 、 100 人 以上 が 負傷 し た 。
Predicted: O O O O O O O O O O O O O O O O O O O O
Correct:   O O O O O O O O O O O O O O O O O O O O
BBC の 報道 に よれ ば 、 少なくとも 49 人 が 死亡 し た 。
Predicted: B-ORG O O O O O O O O O O O O O O
Correct:   B-ORG O O O O O O O O O O O O O O
ロイター通信 社 報道局 は 、 現場 に い た 複数 の 医師 が 、 負傷 者 の 多く は 危険 な 状態 に ある と 語っ て いる と 伝え た 。
Predicted: B-ORG I-ORG I-ORG O O O O 

IndexError: list index out of range

In [15]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

履 歴 書 ふ り が な さ と う い ち ろ う 名 前 砂 糖 一 郎 2 0 1 9 年 1 2 月 2 5 日 現 在 1 9 9 5 年 1 月 1 日 生 （ 満 2 4 歳 ） 男 ・ 女 ふ り が な 現 住 所 と う き ょ う と ち よ だ く さ ざ ん た わ ー 〒 1 0 0 - 0 0 0 0 東 京 都 千 代 田 区 サ ザ ン タ ワ ー ふ り が な 現 住 所 〒 ( 現 住 所 以 外 に 連 絡 を 希 望 す る 場 合 の み 入 力 ) 電 話 E m a i l 証 明 写 真 縦 3 6 m m 〜 4 0 m m 横 2 4 m m 〜 3 0 m m 本 人 単 身 胸 か ら 上 裏 面 の り づ け 電 話 0 0 0 - 0 0 0 0 - 0 0 0 0 E m a i l t e s t 1 @ s a m p l e . c o m 年 月 2 0 1 0 2 0 1 3 2 0 1 3 2 0 1 7 4 3 4 3 東 京 都 砂 糖 高 等 学 校 入 学 東 京 都 砂 糖 高 等 学 校 卒 業 東 京 都 菓 子 大 学 入 学 東 京 都 菓 子 大 学 卒 業 2 0 1 7 1 0 D o n u t s 株 式 会 社 入 社 現 在 に 至 る 学 歴 ・ 職 歴 学 歴 職 歴 以 上 年 月 学 歴 ・ 職 歴 年 2 0 1 2 月 1 2 T O E I C 公 開 テ ス ト 8 0 0 点 取 得 免 許 ・ 資 格 志 望 動 機 ・ 特 技 ・ ア ピ ー ル ポ イ ン ト な ど 通 勤 時 間 D o n u t s は 、 サ ー ビ ス や ゲ ー ム に お い て 1 年 、 2 年 で 終 わ っ て し ま う よ う な も の で は な く 、 世 の 中 に 何 ら か の 価 値 を も た ら し 、 1 0 年 ･ 2 0 年 先 に も そ の 価 値 が 残 り 続 け る も の を 目 指 し て プ ロ ダ ク ト 創 り を し て い ま す 。 そ の た め に 社 会 や 時 流 に 変 化 を も た ら す も の 創 り 出 し て い こ う と い う 扶 養 家 族 数 （ 配 偶 

ValueError: substring not found

In [26]:
s = c.iob_sents('履歴書ふりがなさとういちろう名前砂糖一郎2019年12月25日現在1995年1月1日生（満24歳）男・女ふりがな現住所とうきょうとちよだくさざんたわー〒100-0000東京都千代田区サザンタワーふりがな現住所〒(現住所以外に連絡を希望する場合のみ入力)電話Email証明写真縦36mm〜40mm横24mm〜30mm本人単身胸から上裏面のりづけ電話000-0000-0000Emailtest1@sample.com年月20102013201320174343東京都砂糖高等学校入学東京都砂糖高等学校卒業東京都菓子大学入学東京都菓子大学卒業201710Donuts株式会社入社現在に至る学歴・職歴学歴職歴以上年月学歴・職歴年2012月12TOEIC公開テスト800点取得免許・資格志望動機・特技・アピールポイントなど通勤時間Donutsは、サービスやゲームにおいて1年、2年で終わってしまうようなものではなく、世の中に何らかの価値をもたらし、10年･20年先にもその価値が残り続けるものを目指してプロダクト創りをしています。そのために社会や時流に変化をもたらすもの創り出していこうという扶養家族数（配偶者を除く）約1時間00分電車0人想いを込めてサービスを作り出しています。配偶者有・無配偶者の扶養義務有・無本人希望欄（特に給料・職種・勤務時間・勤務地・その他について希望があれば記入）Donutsは、サービスやゲームにおいて1年、2年で終わってしまうようなものではなく、世の中に何らかの価値をもたらし、10年･20年先にもその価値が残り続けるものを目指してプロダクト創りをしています。そのために社会や時流に変化をもたらすもの創り出していこうという想いを込めてサービスを作り出しています。')

In [27]:
s

In [1]:

import MeCab
mytext = '履歴書ふりがなさとういちろう名前佐藤一郎2019年12月25日現在1995年1月1日生（満24歳）男・女ふりがな現住所とうきょうとちよだくさざんたわー〒100-0000東京都千代田区サザンタワーふりがな現住所〒(現住所以外に連絡を希望する場合のみ入力)電話Email証明写真縦36mm〜40mm横24mm〜30mm本人単身胸から上裏面のりづけ電話000-0000-0000Emailtest1@sample.com年月20102013201320174343東京都砂糖高等学校入学東京都砂糖高等学校卒業東京都菓子大学入学東京都菓子大学卒業201710Donuts株式会社入社現在に至る学歴・職歴学歴職歴以上年月学歴・職歴年2012月12TOEIC公開テスト800点取得免許・資格志望動機・特技・アピールポイントなど通勤時間Donutsは、サービスやゲームにおいて1年、2年で終わってしまうようなものではなく、世の中に何らかの価値をもたらし、10年･20年先にもその価値が残り続けるものを目指してプロダクト創りをしています。そのために社会や時流に変化をもたらすもの創り出していこうという扶養家族数（配偶者を除く）約1時間00分電車0人想いを込めてサービスを作り出しています。配偶者有・無配偶者の扶養義務有・無本人希望欄（特に給料・職種・勤務時間・勤務地・その他について希望があれば記入）Donutsは、サービスやゲームにおいて1年、2年で終わってしまうようなものではなく、世の中に何らかの価値をもたらし、10年･20年先にもその価値が残り続けるものを目指してプロダクト創りをしています。そのために社会や時流に変化をもたらすもの創り出していこうという想いを込めてサービスを作り出しています。'


tagger = MeCab.Tagger()
print(tagger.parse(mytext))

履歴	名詞,一般,*,*,*,*,履歴,リレキ,リレキ
書	名詞,接尾,一般,*,*,*,書,ショ,ショ
ふりがな	名詞,一般,*,*,*,*,ふりがな,フリガナ,フリガナ
さとう	形容詞,自立,*,*,形容詞・アウオ段,連用ゴザイ接続,さとい,サトウ,サトー
いちろう	名詞,固有名詞,人名,名,*,*,いちろう,イチロウ,イチロー
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
佐藤	名詞,固有名詞,人名,姓,*,*,佐藤,サトウ,サトー
一郎	名詞,固有名詞,人名,名,*,*,一郎,イチロウ,イチロー
2019	名詞,数,*,*,*,*,*
年	名詞,接尾,助数詞,*,*,*,年,ネン,ネン
12	名詞,数,*,*,*,*,*
月	名詞,一般,*,*,*,*,月,ツキ,ツキ
25	名詞,数,*,*,*,*,*
日	名詞,接尾,助数詞,*,*,*,日,ニチ,ニチ
現在	名詞,副詞可能,*,*,*,*,現在,ゲンザイ,ゲンザイ
1995	名詞,数,*,*,*,*,*
年	名詞,接尾,助数詞,*,*,*,年,ネン,ネン
1	名詞,数,*,*,*,*,*
月	名詞,一般,*,*,*,*,月,ツキ,ツキ
1	名詞,数,*,*,*,*,*
日	名詞,接尾,助数詞,*,*,*,日,ニチ,ニチ
生	名詞,接尾,一般,*,*,*,生,セイ,セイ
（	記号,括弧開,*,*,*,*,（,（,（
満	接頭詞,数接続,*,*,*,*,満,マン,マン
24	名詞,数,*,*,*,*,*
歳	名詞,接尾,助数詞,*,*,*,歳,サイ,サイ
）	記号,括弧閉,*,*,*,*,）,）,）
男	名詞,一般,*,*,*,*,男,オトコ,オトコ
・	記号,一般,*,*,*,*,・,・,・
女	名詞,一般,*,*,*,*,女,オンナ,オンナ
ふりがな	名詞,一般,*,*,*,*,ふりがな,フリガナ,フリガナ
現住	名詞,サ変接続,*,*,*,*,現住,ゲンジュウ,ゲンジュー
所	名詞,接尾,一般,*,*,*,所,ショ,ショ
とうき	名詞,一般,*,*,*,*,とうき,トウキ,トーキ
ょうとちよだくさざんたわ	名詞,一般,*,*,*,*,*
ー	名詞,一般,*,*,*,*,*
〒	記号,一般,*,*,*,*,〒,ユウビンバンゴウ,ユービンバンゴー
100	名詞,数,*,*,*,*,*
-	名詞,サ